In [6]:
from __future__ import print_function
from six.moves import range
from PIL import Image

import sys
dir_path = '/home/avoyd/GANtor-Arts-Center/src/code/main.py'
sys.path.append(dir_path)
sys.path.append('/home/avoyd/GANtor-Arts-Center/src/code/')

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import os
import time

import numpy as np
import torchfile

from miscc.config import cfg, cfg_from_file
from miscc.utils import mkdir_p
from miscc.utils import weights_init
from miscc.utils import save_img_results, save_model
from miscc.utils import KL_loss
from miscc.utils import compute_discriminator_loss, compute_generator_loss

from tensorboard import summary
from tensorboardX import FileWriter

import torchvision.utils as vutils


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device(0)
print(torch.cuda.is_available())

True


In [8]:
cfg_from_file('../cfg/wikiart_s2_style.yml')

from model import STAGE1_G, STAGE2_G, STAGE2_D

Stage1_G = STAGE1_G()

stage = 1
category = "style"
image_size = 64 if stage == 1 else 256
image_dir = './baseline_generated/{}{}/'.format(category, image_size)

style_1 = '../../../results/wikiart_stageI_2019_05_14_23_03_43/Model/netG_epoch_60.pth'
style_2 = '../../../results/wikiart_stageII_2019_05_15_03_54_54/Model/netG_epoch_45.pth'
stage_1_file = style_1
stage_2_file = style_2

if stage == 1:
    netG = Stage1_G
    state_dict = torch.load(stage_1_file,map_location=lambda storage, loc: storage)
    netG.load_state_dict(state_dict)
    print('Load from: ', stage_1_file)
    
elif stage == 2:
    netG = STAGE2_G(Stage1_G)
    state_dict = torch.load(stage_1_file,map_location=lambda storage, loc: storage)
    netG.STAGE1_G.load_state_dict(state_dict)
    print('Load from: ', stage_1_file)
    state_dict = torch.load(stage_2_file, map_location=lambda storage, loc: storage)
    netG.load_state_dict(state_dict)
    print('Load from: ', stage_2_file)

else:
    raise Exception ("Stage unspecified!")
    
netG.eval()

if cfg.CUDA:
    netG.cuda()

print(netG)


Load from:  ../../../results/wikiart_stageI_2019_05_14_23_03_43/Model/netG_epoch_60.pth
STAGE1_G(
  (fc): Sequential(
    (0): Linear(in_features=127, out_features=24576, bias=False)
    (1): BatchNorm1d(24576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (upsample1): Sequential(
    (0): Upsample(scale_factor=2, mode=nearest)
    (1): Conv2d(1536, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace)
  )
  (upsample2): Sequential(
    (0): Upsample(scale_factor=2, mode=nearest)
    (1): Conv2d(768, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace)
  )
  (upsample3): Sequential(
    (0): Upsample(scale_factor=2, mode=nearest)
    (1): Conv2d(384, 192, kernel_size=(3, 3), stride=(1, 1),

In [9]:
nz = 100
batch_size = 16
embedding_size = 27 if category == "style" else 10
num_iters = 50 #50 * 27 * 16 = 21600 images

for i in range(num_iters):
    print ("Iter num %i"%(i))
    for class_idx in range(embedding_size):
        noise = Variable(torch.FloatTensor(batch_size, nz))
        with torch.no_grad():
                    fixed_noise = \
                        Variable(torch.FloatTensor(batch_size, nz).normal_(0, 1))
        noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

        noise.data.normal_(0, 1)
        text_embeddings = torch.zeros((batch_size, embedding_size)).cuda()
        text_embeddings[:, class_idx] = 1

#         inputs = (text_embeddings, noise)

        lr_fake, fake = netG(text_embeddings, noise)

        for im_idx, im in enumerate(fake.data):
            
            vutils.save_image(
                        im, '%s%i/%i_%i.png' %
                        (image_dir, class_idx, i, im_idx), normalize=True)

Iter num 0


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Iter num 1
Iter num 2
Iter num 3
Iter num 4
Iter num 5
Iter num 6
Iter num 7
Iter num 8
Iter num 9
Iter num 10
Iter num 11
Iter num 12
Iter num 13
Iter num 14
Iter num 15
Iter num 16
Iter num 17
Iter num 18
Iter num 19
Iter num 20
Iter num 21
Iter num 22
Iter num 23
Iter num 24
Iter num 25
Iter num 26
Iter num 27
Iter num 28
Iter num 29
Iter num 30
Iter num 31
Iter num 32
Iter num 33
Iter num 34
Iter num 35
Iter num 36
Iter num 37
Iter num 38
Iter num 39
Iter num 40
Iter num 41
Iter num 42
Iter num 43
Iter num 44
Iter num 45
Iter num 46
Iter num 47
Iter num 48
Iter num 49
